In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyspark

In [ ]:
#Pregunta 1
#De acuerdo a lo enunciado en el ejercicio del caso 2, se asume que el dataset del mes de Marzo y Abril
#corresponden a pruebas molecuales de Marzo y Abril respectivamente
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SparkSession 
sc = SparkContext.getOrCreate() 
spark = SparkSession(sc) 
df_marzo = spark.read.load("../input/prueba-molecular-2021-3/pm25Marzo2021.csv", format="csv", sep="|",header="true",inferSchema="true" )\
.toDF("FECHATOMAMUESTRA", "tipomuestra", "RESULTADO", "edadpaciente_c", "sexopaciente", "Institucion", "DepOrigen", "ProvOrigen", "Disa_DiresaOrigen", "RedOrigen")
df_abril = spark.read.load("../input/prueba-molecular-2021-4/pm27Abril2021.csv", format="csv", sep="|",header="true",inferSchema="true" )\
.toDF("FECHATOMAMUESTRA", "tipomuestra", "RESULTADO", "edadpaciente_c", "sexopaciente", "Institucion", "DepOrigen", "ProvOrigen", "Disa_DiresaOrigen", "RedOrigen")

In [ ]:
#Pregunta 2
p2=spark.sql("select 'MARZO' MES,count(*) CANTIDAD_PRUEBAS from molecular_marzo group by 'MARZO' \
           UNION ALL \
           select 'ABRIL' MES,count(*) CANTIDAD_PRUEBAS from molecular_abril group by 'ABRIL'")
p2.show()
p2_panda=p2.toPandas()
sns.barplot(x='MES',y='CANTIDAD_PRUEBAS',data=p1_panda)
plt.show()

In [ ]:
#Pregunta 3
p3=spark.sql("Select MES,SEXOPACIENTE,count(*) CANTIDAD_PRUEBAS From (\
             select 'MARZO' MES,SEXOPACIENTE from molecular_marzo where sexopaciente<>'null'\
           UNION ALL \
           select 'ABRIL' MES,SEXOPACIENTE from molecular_abril where sexopaciente<>'null') \
           group by MES,SEXOPACIENTE order by MES DESC,CANTIDAD_PRUEBAS DESC")
p3.show()
p3_panda=p3.toPandas()
sns.barplot(x='MES',y='CANTIDAD_PRUEBAS',hue='SEXOPACIENTE',data=p3_panda)
plt.show()

In [ ]:
#Pregunta 4
p4=spark.sql("Select MES,DEPARTAMENTO,count(*) CANTIDAD_PRUEBAS From (\
             select 'MARZO' MES,DepOrigen DEPARTAMENTO from molecular_marzo where sexopaciente<>'null' \
           UNION ALL \
           select 'ABRIL' MES,DepOrigen DEPARTAMENTO from molecular_abril where sexopaciente<>'null') \
           group by MES,DEPARTAMENTO order by MES desc,CANTIDAD_PRUEBAS desc")
p4.show(70)
p4_panda=p4.toPandas()
fig, ax = plt.subplots(figsize = (20,10))
sns.barplot(x='CANTIDAD_PRUEBAS',y='DEPARTAMENTO',hue='MES',data=p4_panda)
plt.show()

In [ ]:
#Pregunta 5
p5_marzo=df_marzo.filter(df_marzo.RESULTADO=="POSITIVO").count()
p5_abril=df_abril.filter(df_abril.RESULTADO=="POSITIVO").count()

print("positivos en marzo:",p5_marzo)
print("positivos en abril:",p5_abril)

#p5=spark.sql("select 'MARZO' MES,count(*) CANTIDAD_PRUEBAS_POSITIVO from molecular_marzo where trim(resultado) in ('POSITIVO','POSTIVO') group by 'MARZO' \
#          UNION ALL \
#          select 'ABRIL' MES,count(*) CANTIDAD_PRUEBAS_POSITIVO from molecular_abril where trim(resultado) in ('POSITIVO','POSTIVO') group by 'ABRIL'")
#p5.show()
#p5_panda=p5.toPandas()
#sns.barplot(x='MES',y='CANTIDAD_PRUEBAS_POSITIVO',data=p5_panda)
#plt.show()

In [ ]:
#Pregunta 6
p6_marzo=df_marzo.filter(df_marzo.RESULTADO=="NEGATIVO").count()
p6_abril=df_abril.filter(df_abril.RESULTADO=="NEGATIVO").count()

print("negativos en marzo:",p6_marzo)
print("negativos en abril:",p6_abril)

#p6=spark.sql("select 'MARZO' MES,count(*) CANTIDAD_PRUEBAS_NEGATIVO from molecular_marzo where trim(resultado) in ('NEGATIVO','NEGTIVO') group by 'MARZO' \
#          UNION ALL \
#          select 'ABRIL' MES,count(*) CANTIDAD_PRUEBAS_NEGATIVO from molecular_abril where trim(resultado) in ('NEGATIVO','NEGTIVO') group by 'ABRIL'")
#p6.show()
#p6_panda=p6.toPandas()
#sns.barplot(x='MES',y='CANTIDAD_PRUEBAS_NEGATIVO',data=p6_panda)
#plt.show()